In [1]:
import pandas as pd

In [5]:
kepler_tce_df = pd.read_csv("/Users/derrickvanfrausum/BeCode_AI/git-repos/astro/core/assets/data/Kepler/thread_csv_all.csv")
kepler_tce_df.head()

,time,flux,kepid,best_fit_period,av_training_set
0,-3.917173,1.000192,11551692,7.836,PC
1,-3.917173,1.000192,11551692,7.836,PC
2,-3.917173,1.000192,11551692,7.836,PC
3,-3.912650,0.999829,11551692,7.836,PC
4,-3.912650,0.999829,11551692,7.836,PC


In [6]:
kepler_tce_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059996 entries, 0 to 6059995
Data columns (total 5 columns):
 #   Column           Dtype  
---  ------           -----  
 0   time             float64
 1   flux             float64
 2   kepid            int64  
 3   best_fit_period  float64
 4   av_training_set  object 
dtypes: float64(3), int64(1), object(1)
memory usage: 231.2+ MB


In [9]:
kepler_tce_df[["kepid", "time"]].groupby('kepid').count()

,time
kepid,
11200767,63794
11201483,31494
11206101,51710
11228612,122246
11231334,103942
...,...
11568624,92744
11568987,129582
11569475,130492


In [11]:
# Create Function to explore dataframes
def explore(df: pd.DataFrame) -> None:
    """
    Function to print general information about the dataframe
    """
    print("********** 1. General info of data **********")
    print(df.info())
    
    print("\n********** 2. Shape of data **********")
    print(f"Number of rows: {len(df)}")
    print(f"Number of columns: {len(df.columns)}")
    
    print("\n********** 3. Number of missing values per column **********")
    print(df.isnull().sum())
    
    print("\n********** 4. Number of duplicated values **********")
    print(df.duplicated().sum())
    
    print("\n********** 5. Number of unique values per column (NaN non included) **********")
    print(df.nunique())
    
    print("\n********** 6. Statistical info of each column **********")
    print(df.describe(include='all').T)

In [12]:
explore(kepler_tce_df)

********** 1. General info of data **********
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6059996 entries, 0 to 6059995
Data columns (total 5 columns):
 #   Column           Dtype  
---  ------           -----  
 0   time             float64
 1   flux             float64
 2   kepid            int64  
 3   best_fit_period  float64
 4   av_training_set  object 
dtypes: float64(3), int64(1), object(1)
memory usage: 231.2+ MB
None

********** 2. Shape of data **********
Number of rows: 6059996
Number of columns: 5

********** 3. Number of missing values per column **********
time               0
flux               0
kepid              0
best_fit_period    0
av_training_set    0
dtype: int64

********** 4. Number of duplicated values **********
1923360

********** 5. Number of unique values per column (NaN non included) **********
time               3662900
flux               3662327
kepid                   63
best_fit_period        538
av_training_set          4
dtype: int64

*******

In [13]:
(0.5 + 0.808 + 1.49 + 5.317)/4

2.02875